<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/eval_ENEM_2022_with_HF_and_chatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets transformers rank_bm25 openai accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# ---------------------------------------------------------- #
# This code is from https://github.com/piresramon/gpt-4-enem #
# ---------------------------------------------------------- #

import json
import torch
import random 
import collections

import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('max_colwidth', 400)

from tqdm.auto import tqdm

import openai

import rank_bm25

from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score
from datasets import Dataset

if torch.cuda.is_available(): 
    device = 'cuda'  
else: 
    device ='cpu'

MANUAL_SEED = 2711
rnd = random.Random()
rnd.seed(MANUAL_SEED)
def deterministic(rep=True, manual_seed=MANUAL_SEED):
    if rep:
        torch.manual_seed(manual_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(manual_seed)
            torch.cuda.manual_seed_all(manual_seed)
        torch.backends.cudnn.enabled = False 
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        print(f'Experimento deterministico, seed: {manual_seed}')
        if device == 'cuda':
            print(f'Existe {torch.cuda.device_count()} GPU\
            {torch.cuda.get_device_name(0)} disponível.')
    else:
        print('Experimento randomico')
deterministic()    

Experimento deterministico, seed: 2711
Existe 1 GPU            Tesla T4 disponível.


# Get Enem json and create enem_data 

In [58]:
def ignore_question(doc):
    filters = {
        'IU': False,
        # 'MR': False,  # uncomment to filter out MR
        # 'CE': False,  # uncomment to filter out CE
        'ML': False,
    }
    for k,v in filters.items():
        if doc[k] != v:
            return True
    return False

def _process_doc_cot(doc):
    def format_example(doc, choices):
        prompt = "Contexto: " + doc["context"] + "\n"
        prompt += "Enunciado: " + doc["question"] + "\nAlternativas:\n"
        for choice, option in zip(choices, doc["options"]):
            prompt += f"{choice.upper()}. {option}\n"
        
        prompt += "Explicação: " + doc.get("explanation", "")
        return prompt.strip()
    choices = ['a', 'b', 'c', 'd', 'e']
    return {
        "query": format_example(doc, choices),
        "choices": doc["options"],
        "gold": choices.index(doc["label"]),
        "id": doc["id"],
        "exam": doc["exam"],
    }    

dataset = collections.defaultdict(list)
        
data_path = "/content/drive/MyDrive/LLMs/ENEM/ENEMdataset/2022.json"
with open(data_path) as f:
    documents = json.load(f)

documents = list(filter(lambda doc: not ignore_question(doc), documents))
dataset['test'] = list(map(_process_doc_cot, documents))

enem_data = {ix:doc  for ix, doc in enumerate(dataset['test'])}

print(f"tamanho dataset: {len(enem_data)}")

enem_data[0]    

tamanho dataset: 118


{'query': 'Contexto: A conquista da medalha de prata por Rayssa Leal, no skate street nos Jogos Olímpicos, é exemplo da representatividade feminina no esporte, avalia a âncora do jornal da rede de televisão da CNN. A apresentadora, que também anda de skate, celebrou a vitória da brasileira, que entrou para a história como a atleta mais nova a subir num pódio defendendo o Brasil. “Essa representatividade do esporte nos Jogos faz pensarmos que não temos que ficar nos encaixando em nenhum lugar. Posso gostar de passar notícia e, mesmo assim, gostar de skate, subir montanha, mergulhar, andar de bike, fazer yoga. Temos que parar de ficar enquadrando as pessoas dentro de regras. A gente vive num padrão no qual a menina ganha boneca, mas por que também não fazer um esporte de aventura? Por que o homem pode se machucar, cair de joelhos, e a menina tem que estar sempre lindinha dentro de um padrão? Acabamos limitando os talentos das pessoas”, afirmou a jornalista, sobre a prática do skate por m

# Create Prompts
- #### Dynamic Prompt (fewshot sampled)
- #### Dynamic Similar Prompt (bm25 rank fewshot)

In [61]:
def dynamic_similar_prompt(doc_id, data, topk=3):
    key_predict = [k for k, v in enem_data.items() if v['id'].startswith(doc_id)][0]
    query_list = [v['query'] for k,v in data.items()]
    tokenized_corpus = [doc.split(" ") for doc in query_list]
    bm25 = rank_bm25.BM25Plus(tokenized_corpus)

    query = data[key_predict]['query']
    tokenized_query = query.split(" ")
    
    doc_scores = bm25.get_scores(tokenized_query)
    
    # [1:] remove query with itself ---> topk+1
    top_similar_idxs = list(doc_scores.argsort()[-(topk+1):][::-1])[1:]
    if key_predict in top_similar_idxs: # k in topk_sim ---> must be false
        print('Error BM25 prompt similar retrieval')
    
    return top_similar_idxs


def dynamyc_fewshot_examples(num_fewshot, doc_id, sim_prompt=True, data=enem_data):
    if sim_prompt:
        topk_sim = dynamic_similar_prompt(doc_id, data, topk=num_fewshot)
        fewshot_ex = [enem_data[k] for k in topk_sim]

    else:
        # filter the sample in current predict from data
        all_possible_fewshot_keys = [k for k, v in data.items() if not v['id'].startswith(doc_id)]
        fewshot_keys = rnd.sample(all_possible_fewshot_keys, num_fewshot)
        fewshot_ex = [enem_data[k] for k in fewshot_keys]

    return fewshot_ex


def fewshot_context(doc, num_fewshot, similar_prompt=True, enem_data=enem_data):
    if num_fewshot == 0 and fewshotex is None:
        labeled_examples = ""
    else:
        fewshotex = dynamyc_fewshot_examples(
            num_fewshot=num_fewshot, doc_id=doc['id'], sim_prompt=similar_prompt, data=enem_data
        )
        
        labeled_examples = "Dado o exemplo, forneça uma resposta após ##Explicação:## não faça nenhuma explicação adicional" + "\n"
        for i, doc_ex in enumerate(fewshotex):
            labeled_examples += f'Questão {i+1}:\n'
            labeled_examples += doc_ex['query'] + " " + ['A.', 'B.', 'C.', 'D.', 'E.'][doc_ex['gold']].upper()
            labeled_examples += '\n##\n'
        labeled_examples += f'Questão {len(fewshotex) + 1}:\n'

    example = doc['query']
    
    return labeled_examples + example

# -------------------------------------------------------------------------------------- #
# number of fewshots, must be > 0

# create inputs ---> list with fewshot samples + question to be answer 
one_shot, two_shot, three_shot = [],[],[]
for k,doc in enem_data.items():
    one_shot.append(fewshot_context(doc=doc,   num_fewshot=1, similar_prompt=False, enem_data=enem_data))
    two_shot.append(fewshot_context(doc=doc,   num_fewshot=2, similar_prompt=False, enem_data=enem_data))
    three_shot.append(fewshot_context(doc=doc, num_fewshot=3, similar_prompt=False, enem_data=enem_data))

In [62]:
one_shot[0]

'Dado o exemplo, forneça uma resposta após ##Explicação:## não faça nenhuma explicação adicional\nQuestão 1:\nContexto: Um prédio, com 9 andares e 8 apartamentos de 2 quartos por andar, está com todos os seus apartamentos à venda. Os apartamentos são identificados por números formados por dois algarismos, sendo que a dezena indica o andar onde se encontra o apartamento, e a unidade, um algarismo de 1 a 8, que diferencia os apartamentos de um mesmo andar. Quanto à incidência de sol nos quartos desses apartamentos, constatam-se as seguintes características, em função de seus números de identificação: • n aqueles que finalizam em 1 ou 2, ambos os quartos recebem sol apenas na parte da manhã; • n aqueles que finalizam em 3, 4, 5 ou 6, apenas um dos quartos recebe sol na parte da manhã; • n aqueles que finalizam em 7 ou 8, ambos os quartos recebem sol apenas na parte da tarde. Uma pessoa pretende comprar 2 desses apartamentos em um mesmo andar, mas quer que, em ambos, pelo menos um dos quar

# Eval ENEM-2022 with ChatGPT 

In [76]:
EVAL_CHATGPT = True

if EVAL_CHATGPT:
    # OPENAI_API_KEY = "secret"
    # openai.api_key = OPENAI_API_KEY

    deterministic()    

    trues, preds = [], []
    loop = tqdm(one_shot, leave=True)

    for ix, batch in enumerate(loop):

        # to make sure the same example to predict has the correct label 
        # find the point where the question to be evaluate starts
        point = one_shot[ix].find('Questão 2:\n')
        
        # "11+point" ---> is the start of the query-text, so this is a naive approach
        if enem_data[ix]['query'] == one_shot[ix][11+point:]:
            chatGPT_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": one_shot[ix]}])
            
            preds.append(chatGPT_response['choices'][0]['message']['content'])
            trues.append(enem_data[ix]['gold'])
        else:
            # if the query-text is different from inputs to be evaluated 
            # we print for further investigation
            print(point, ix)

Experimento deterministico, seed: 2711
Existe 1 GPU            Tesla T4 disponível.


  0%|          | 0/118 [00:00<?, ?it/s]

In [80]:
dataframe = pd.DataFrame({'true': trues, 'pred':preds})
dataframe.pred.apply(lambda x: x[-2:-1])

0      C
1      C
2      C
3      D
4      C
5      C
6      A
7      B
8      E
9      C
10     B
11     A
12     E
13     E
14     C
15     A
16     B
17     A
18     E
19     E
20     B
21     E
22     E
23     D
24     C
25     A
26     D
27     B
28     D
29     B
30     D
31     B
32     A
33     C
34     A
35     E
36     C
37     A
38     B
39     C
40     D
41     A
42     C
43     D
44     E
45     D
46     A
47     D
48     D
49     B
50     B
51     D
52     A
53     E
54     C
55     D
56     C
57     A
58     E
59     C
60     B
61     E
62     B
63     E
64     E
65     D
66     B
67     D
68     E
69     A
70     D
71     E
72     D
73     B
74     E
75     C
76     A
77     E
78     B
79     B
80     B
81     C
82     D
83     C
84     B
85     D
86     E
87     E
88     A
89     D
90     E
91     B
92     E
93     D
94     C
95     A
96     E
97     C
98     B
99     D
100    E
101    B
102    A
103    D
104    D
105    D
106    E
107    C
108    D
109    D
110    D
1

In [81]:
def get_results(trues, preds, chatgpt=False):
    dataframe = pd.DataFrame({'true': trues, 'pred':preds})

    #get only latter
    if chatgpt:
        dataframe['pred'] = dataframe.pred.apply(lambda x: x[-2:-1])
    
    else:
        dataframe['pred'] = dataframe.pred.apply(lambda x: x[-1:])
    
    map_label = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E"}
    dataframe['true'] = dataframe.true.apply(lambda x: map_label[x])
    
    acc = accuracy_score(dataframe['true'], dataframe['pred'])
    
    return dataframe, acc


if EVAL_CHATGPT:
    chatGPT_results, acc = get_results(trues, preds, chatgpt=True)

    print(f'ACC of 1-shot chatGPT on ENEM-2022: {acc:.3}')
    # ACC of ChatGPT: 0.771

    chatGPT_results

ACC of 1-shot chatGPT on ENEM-2022: 0.754


# Open Source Models

- #### with HuggingFace we will evaluate GPT2/LLama & Alpaca/Bloom models.

In [7]:
model_gpt = 'gpt2'
tokenizer_gpt = AutoTokenizer.from_pretrained(model_gpt)

model_bloom = 'bigscience/bloomz-7b1-mt'
tokenizer_bloom = AutoTokenizer.from_pretrained(model_bloom)

def get_input_lenghts(dataset, tokenizer):
    num_words = [len(x.split()) for x in dataset]
    num_words = torch.tensor(num_words, dtype=torch.float)
    tokens_lengths = [len(tokenizer.encode(seq)) for seq in dataset]
    tokens_lengths = torch.tensor(tokens_lengths, dtype=torch.float)
    
    return int(torch.ceil(num_words.mean()).item()), tokens_lengths

# -------------------------------------------------------------------- #
num_words, gpt2_lengths = get_input_lenghts(three_shot, tokenizer_gpt)
min_gpt2 = torch.ceil(gpt2_lengths.min()).item()
max_gpt2 = torch.ceil(gpt2_lengths.max()).item()
mean_gpt2 = torch.ceil(gpt2_lengths.mean()).item()
# -------------------------------------------------------------------- #
num_words, bloom_lengths = get_input_lenghts(three_shot, tokenizer_bloom)
min_bloom = torch.ceil(bloom_lengths.min()).item()
max_bloom = torch.ceil(bloom_lengths.max()).item()
mean_bloom = torch.ceil(bloom_lengths.mean()).item()
# -------------------------------------------------------------------- #
print('\n\n')
print('--'*33)
print(f' Prompt 3-shot com média de {num_words} palavras em  ENEM2022 dataset')
print('--'*33)
print(f'GPT2 tem média de {int(mean_gpt2)} tokens em 3-shot')
print(f'\nBloom tem média de {int(mean_bloom)} tokens em 3-shot')
print(f'\n---> Bloom gasta {(int(mean_bloom)/int(mean_gpt2))*100:.3}% de tokens comparado com o GPT2 <---')

Token indices sequence length is longer than the specified maximum sequence length for this model (2863 > 1024). Running this sequence through the model will result in indexing errors





------------------------------------------------------------------
 Prompt 3-shot com média de 862 palavras em  ENEM2022 dataset
------------------------------------------------------------------
GPT2 tem média de 2061 tokens em 3-shot

Bloom tem média de 1245 tokens em 3-shot

---> Bloom gasta 60.4% de tokens comparado com o GPT2 <---


# Eval BloomZ 7b mt

In [8]:
# load bloom in 8bits with bitsandbytes
BLOOM_model = AutoModelForCausalLM.from_pretrained(model_bloom, device_map="auto", load_in_8bit=True)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [63]:
EVAL_OPEN_SOURCE = True

if EVAL_OPEN_SOURCE:
    deterministic()    

    trues, preds = [], []
    loop = tqdm(one_shot, leave=True)

    for ix, batch in enumerate(loop):

        # to make sure the same example to predict has the correct label 
        # find the point where the question to be evaluate starts
        point = one_shot[ix].find('Questão 2:\n')
        
        # "11+point" ---> is the start of the query-text, so this is a naive approach
        if enem_data[ix]['query'] == one_shot[ix][11+point:]:
            inputs = tokenizer_bloom.encode(one_shot[ix], return_tensors="pt", max_length=2048).to(device)
            outputs = BLOOM_model.generate(inputs)
            
            preds.append(tokenizer_bloom.decode(outputs[0]))
            trues.append(enem_data[ix]['gold'])
        else:
            # if the query-text is different from inputs to be evaluated 
            # we print for further investigation
            print(point, ix)

Experimento deterministico, seed: 2711
Existe 1 GPU            Tesla T4 disponível.


  0%|          | 0/118 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 687, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 688, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 713, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 703, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 566, but `m

In [64]:
dataframe = pd.DataFrame({'true': trues, 'pred':preds})
dataframe.pred.apply(lambda x: x[-1:])

0      D
1      C
2      B
3      D
4      C
5      C
6      A
7      D
8      A
9      C
10     A
11     D
12     A
13     A
14     C
15     A
16     C
17     A
18     A
19     C
20     D
21     A
22     D
23     D
24     D
25     A
26     C
27     A
28     D
29     D
30     D
31     B
32     A
33     D
34     C
35     E
36     C
37     A
38     A
39     A
40     D
41     A
42     D
43     D
44     C
45     D
46     C
47     D
48     D
49     D
50     C
51     D
52     A
53     D
54     C
55     D
56     A
57     A
58     E
59     C
60     B
61     D
62     D
63     A
64     E
65     D
66     B
67     D
68     A
69     A
70     D
71     A
72     C
73     A
74     D
75     C
76     A
77     A
78     B
79     D
80     D
81     D
82     A
83     D
84     D
85     D
86     D
87     E
88     A
89     D
90     D
91     D
92     A
93     D
94     D
95     A
96     D
97     D
98     D
99     D
100    D
101    D
102    A
103    D
104    D
105    D
106    D
107    B
108    D
109    D
110    D
1

In [65]:
bloom_results, acc = get_results(trues, preds)

print(f'ACC of 1-shot Bloomz-7b1-mt on ENEM-2022: {acc:.3}')
# ACC of Bloomz-7b1-mt: 

bloom_results

ACC of 1-shot Bloomz-7b1-mt on ENEM-2022: 0.424


,true,pred
0,C,D
1,C,C
2,C,B
3,D,D
4,C,C
5,C,C
6,E,A
7,B,D
8,A,A
9,C,C
